In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import statistics

from konlpy.tag import Twitter
from konlpy.utils import pprint
from collections import Counter

pd.set_option('max_colwidth', 500)


In [5]:
twitter=Twitter()

In [6]:
 # 소수점 아래 3자리까지 나타내기, 4째자리에서 반올림
def short_float(val):
    value = float("{:.4f}".format(val))
    return value


In [7]:
def print_group(title, groups, used_func):
    pivot_num = 0.1
    save_list = list()

    pprint(title)
    cur_num = 0.0
    for i, group in enumerate(groups):
        save_list.append(used_func(group))
        print('group ' + str(i + 1) + ' = ' + str(float("{:.1f}".format(cur_num))) + str(' over ') + str(
            float("{:.1f}".format(cur_num + pivot_num))) + str(' under : '), save_list[i])
        cur_num += pivot_num
    print('\n')

    return save_list

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [10]:
# tokenizer : 문장에서 색인어 추출을 위해 정해진 품사의 단어만 뽑아서 normalization, stemming 처리하도록 함
def token(raw, pos=["Noun","Alpha","Verb","Number","Adjective","KoreanParticle",
                        "Punctuation","Determiner", "Adverb", "Conjunction","Excalmation", "Foreign"], stopword=[]):
    return [
        word for word, tag in twitter.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
        
          if len(word) >=1 and tag in pos and word not in stopword

        ]



In [11]:
vectorize= TfidfVectorizer(
    tokenizer=token,
    sublinear_tf=True    # tf값에 1+log(tf)를 적용하여 tf값이 무한정 커지는 것을 막음
)

In [12]:
# 1.
#reviewer의 desc를 담는 리스트
def make_reviews(cid):
    desc=(go_data.loc[go_data['cId']==cid])['desc']
    desc=list(desc)
    
    return desc


In [13]:
#2. 리뷰들을 tf-idf 적용하여 벡터화
def tfidf(desc):
    X = vectorize.fit_transform(desc)
    print('fit_transform, (No.review {}, feature {})'.format(X.shape[0], X.shape[1]))
    features = vectorize.get_feature_names()
    
    print (pd.DataFrame(data=X.toarray(), columns=features))
    vector_array=X.toarray()
    
    return vector_array

In [16]:
#3. 유사도 구하기222222222
def similarity(vector_arr,desc):   
    sm=[] #한 리뷰어의 리뷰 유사도를 담을 리스트
    max_cnt=0 #리뷰 유사도 최대값이 0.9 이상 count
    _sum=0
    _mid=0
    cnt=0
    for i in range(len(desc)-1):
        srch_vector= vectorize.transform([desc[i]])
        for j in range(i+1, len(desc)):
            cosine_similar =cosine_similarity(srch_vector, [vector_arr[j]]).flatten()
            cosine_similar=short_float(float(cosine_similar))
          #  print ("cosine_similar ") + str(i+1) + str(' 번째 리뷰와') + str(j+1) + str(' 번째 리뷰 : ') + str(cosine_similar)
            sm.append(cosine_similar)


   # pprint(u'유사도 리스트'),;print (sm)

    sm.sort(reverse=True)
    _min=min(sm)
    _mid=statistics.median(sm) # 유사도의 중간값
    _max=max(sm)
    _sum=sum(sm)

    if(_max>=0.9):
        max_cnt+=1
        
    avg= short_float(_sum/len(sm))
    
    
  #  print ('min : ', _min)
  #  print ('mid : ', _mid)
  #  print('max : ', _max)
    print('avg : ', avg)
   # print('num of 1 : ', cnt)
    print('# similarity over 0.9 : ', max_cnt)
    
    return sm


In [14]:
def print_want_val(sm_list,used_func):
    val=(used_func(sm_list))
   
    return val

In [15]:
def make_groups(_list):
    group1=0
    group2=0
    group3=0
    group4=0
    group5=0
    group6=0
    group7=0
    group8=0
    group9=0
    group10=0
    
    for i in range(len(_list)):
        if(_list[i]<0.1):
            group1+=1
        elif(_list[i]>=0.1 and _list[i]< 0.2):
            group2+=1
        elif(_list[i] >=0.2 and _list[i]< 0.3):
            group3+=1
        elif(_list[i] >=0.3 and _list[i]< 0.4):
            group4+=1
        elif(_list[i]>=0.4 and _list[i]< 0.5):
            group5+=1
        elif(_list[i]>=0.5 and _list[i] < 0.6):
            group6+=1
        elif(_list[i]>=0.6 and _list[i] < 0.7):
            group7+=1
        elif(_list[i]>=0.7 and _list[i]< 0.8):
            group8+=1
        elif(_list[i]>=0.8 and _list[i]< 0.9):
            group9+=1
        elif(_list[i]>=0.9):
            group10+=1
        
    groups=[group1, group2, group3, group4, group5, group6,group7,group8,group9,group10 ]
    
    return groups

In [ ]:
max_list=[]
mid_list=[]
q1_list=[]
q3_list=[]

for i in range(len(spammer)):
    print str(i+1) + str('번째 리뷰어: ')+ str(spammer[i])
    reviews=make_reviews(spammer[i])
    if(len(reviews)<2):
        continue
    vector_array=tfidf(reviews)
    sm=similarity(vector_array, reviews)

  
    result_max= print_want_val(sm, lambda x: np.percentile(sm,100)) # 최대값 반환 
    print ("max: ", result_max)
    max_list.append(result_max)
    
    result_mid = print_want_val(sm, lambda x: np.percentile(sm,50)) # 중간값 반환 
    print ("median: ", result_mid)
    mid_list.append(result_mid)
    
    result_q1 = print_want_val(sm, lambda x: np.percentile(sm,25)) # 상위 75%
    print("1st quartile: ", result_q1)
    q1_list.append(result_q1)
    
    result_q3 = print_want_val(sm, lambda x: np.percentile(sm,75)) #상위 25%
    print("3rd quartile: ", result_q3)         
    q3_list.append(result_q3)


    print("\n")
    
groups_max=make_groups(max_list)
groups_mid=make_groups(mid_list)
groups_q1=make_groups(q1_list)
groups_q3=make_groups(q3_list)